In [1]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore, SPARQLStore
from rdflib import Graph, Namespace, URIRef

from xml.dom.minidom import parse
import xml.etree.ElementTree as etree
from Marc_to_Bibframe.Marc.marcWork import MarcWork
from Marc_to_Bibframe.Marc.marcInstance import MarcInstance
from Marc_to_Bibframe.Marc.marcItems import MarcItems
from Marc_to_Bibframe.Work.work import Work
from Marc_to_Bibframe.Instance.instance import Instance

import pysolr
from Solr.solr import create_doc
#from Jena.jena import create_graph

JENA

In [6]:
#JENA
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/bibframe/update')
query_endpoint = 'http://localhost:3030/bibframe/query'
update_endpoint = 'http://localhost:3030/bibframe/update'
store.open((query_endpoint, update_endpoint))

DELETE ALL GRAPHS

In [7]:
#Delete all Named Graph

d = """DELETE { graph ?g { ?s ?p ?o } } 

WHERE {

graph ?g {?s ?p ?o.}

}"""

store.update(d)

SOLR

In [3]:
#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":201,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-3"}},\n  "status":"OK"}\n'

In [8]:
def Make_Graph(nt, bf, count ):
    G1 = "PREFIX bk: <http://bibliokeia.com/bibframe/"+bf+"""/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {
        GRAPH bk:"""

    G2 = " {"

    G3 = """}
    }"""

    G = G1+str(count)+" { \n"+nt+"}}"

    return G

In [6]:
def Upadate(path_marc, count, shelf, out):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(count)
        print(workMarc.Title().get('title'))

        #SOLR
        doc = create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)

        #JENA
        #create_graph(count, workMarc, instanceMarc, itemsMarc, shelf)
 

        count += 1
    solr.add(docs, commit=True)

In [2]:
def UpadateSolr(path_marc, count, shelf, out):

    #SOLR
    #solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
    #solr.ping()
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(count)
        print(workMarc.Title().get('title'))

        #SOLR
        doc = create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)
        
        count += 1
    return docs

    #solr.add(docs, commit=True)

In [9]:
count = 180
e = '2'
p = '3'
out = f'E{e}/P{p}'
path_marc = f'Input/{out}/koha.xml'

docs = UpadateSolr(path_marc, count, f"E{e}.P{p}", out)
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.add(docs, commit=True)

180
Needs and specifications for biodiversity information network
181
Vampiros extraterrestres na Amazônia
182
Normas para publicação de trabalhos técnicos-científicos do CPPSE
183
Manual de walkthroughs
184
Manual de desenvolvimento de sistemas estruturados
185
Bioestat
186
Aide-mémoire d'informatique
187
Periódicos brasileiros em microformas
188
Coleção Affonso Penna Junior
189
Catálogo de obras raras sobre a Amazônia (1800 - 1899)
190
Catálogo de serviços
191
Plano anual de trabalho 1994
192
Educação, ciência e tecnologia
193
The Brazilian Amazon
194
Catálogo da coleção Béatrix Reynal
195
Catálogo de dissertação e teses dos técnicos do incaper
196
Resumos de teses e dissertações do Projeto Dinâmica Biológica de Fragmentos Florestais
197
Catálogo de teses e dissertações da Universidade do Amazonas (1952-1985)
198
Catálogo de teses e dissertações da Universidade do Amazonas (1989 - 1993)
199
Cem anos de imprensa no Amazonas (1851-1951)
200
Catálogo de teses e dissertações da Universid

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":602}}\n'

In [12]:
def UpadateJena(path_marc, count, shelf, out):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(count)
        print(workMarc.Title().get('title'))

        #JENA
        create_graph(count, workMarc, instanceMarc, itemsMarc, shelf)
        count += 1

In [13]:
count = 20
e = '1'
p = '2'
out = f'E{e}/P{p}'
path_marc = f'Input/{out}/koha.xml'

UpadateJena(path_marc, count, f"E{e}.P{p}", out)

20
Resumos
21
Resumos
22
Pesquisa científica e tecnológica em saúde
23
Pesquisas recentes em energia, meio ambiente e tecnologia
24
Iniciação à pesquisa científica
25
<1º >Seminário de Ciências da Fiube
26
Encontro sobre o programa de pesquisas para o trópico úmido.
27
A pesquisa no Brasil


Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x0000025BF6578790>
Traceback (most recent call last):
  File "c:\Users\Usuário\Desktop\BiblioKeia\venv\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "c:\Users\Usuário\Desktop\BiblioKeia\venv\lib\site-packages\isodate\isodates.py", line 201, in parse_date
    return date(sign * int(groups['year']),
ValueError: year 0 is out of range
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x0000025BF6578790>
Traceback (most recent call last):
  File "c:\Users\Usuário\Desktop\BiblioKeia\venv\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "c:\Users\Usuário\Desktop\BiblioKeia\venv\lib\site-packages\isodate\isodates.py

28
Resumos do XVII Seminário de Iniciação Científica da UFPA
29
FADESP, Fundação de Amaparo e Desenvolvimento da Pesquisa, Belém-Pará.
30
Manual de normas para publicações técnico-científicas
31
Manual do autor
32
Metodos e tecnicas de pesquisa aplicados a administração
33
A estatística na pesquisa científica
34
Prioridades da pesquisa do CIFOR
35
Matemática e estatística na análise de experimentos e no melhoramento genético
36
Relatório de atividades 2011-2013
37
Seminário pesquisa tecnológica
38
Pesquisa científica
39
Pesquisa operacional
40
Ciência, tecnologia e inovação para um Brasil competitivo
41
Os 40 anos da fundação Ford no Brasil
42
Orçamento da União para ciência e tecnologia, 1980
43
Guia de fontes de financiamento à ciência & tecnologia
44
Guia de fontes de financiamento à ciência & tecnologia
45
Avaliação e fomento de C&T no Brasil
46
Recherches de I'RD au Brésil depuis 1998
47
Setor produtivo estatal
